In [1]:
!pip install numpy > 2
# !pip install torch

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'CPU'
print(device)
block_size = 8
batch_size = 4
learning_rate = 3e-4
max_iters = 10000
eval_iters = 250

CPU


In [3]:
with open('wizard_of_oz.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()
    
print(len(text))
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

232314
['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [4]:
string_to_int = {ch:i for i, ch in enumerate(chars)}
int_to_string = {i:ch for i, ch in enumerate(chars)}
encode = lambda s : [string_to_int[j] for j in s]
decode  = lambda l: ''.join([int_to_string[i] for i in l])

In [5]:
encode('hello')

[61, 58, 65, 65, 68]

In [6]:
decode(encode('hello'))

'hello'

In [7]:
data = torch.tensor(encode(text), dtype = torch.long)

In [8]:
data

tensor([80, 28, 39,  ...,  1,  1,  0])

In [9]:
print(data[:100])

tensor([80, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,  0,
         0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0,
         1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33,
        50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25,
        38, 28,  1, 39, 30,  1, 39, 50,  9,  1])


In [10]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(n-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x , y = x.to(device), y.to(device)
    return x , y

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]


In [12]:
for i in range(block_size):
    context = x[:i+1]
    target = y[i]
    print("When the input is ",context, " target is ", target)

When the input is  tensor([80])  target is  tensor(28)
When the input is  tensor([80, 28])  target is  tensor(39)
When the input is  tensor([80, 28, 39])  target is  tensor(42)
When the input is  tensor([80, 28, 39, 42])  target is  tensor(39)
When the input is  tensor([80, 28, 39, 42, 39])  target is  tensor(44)
When the input is  tensor([80, 28, 39, 42, 39, 44])  target is  tensor(32)
When the input is  tensor([80, 28, 39, 42, 39, 44, 32])  target is  tensor(49)
When the input is  tensor([80, 28, 39, 42, 39, 44, 32, 49])  target is  tensor(1)


In [13]:

import torch
import torch.nn as nn
import torch.nn.functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.vocab_size = vocab_size
        self.token_embedding_table = nn.Embedding(self.vocab_size, self.vocab_size)
    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else: 
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
        
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, _ = self.forward(index)
            logits = logits[:, -1:, :]  # Consider only the last time step's logits
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs.squeeze(1), num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype = torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens = 500)[0].tolist())
print(m.generate(context, max_new_tokens = 500))
print(m.generate(context, max_new_tokens = 500)[0])
print(generated_chars)      
            
            
        
                


RuntimeError: Expected one of cpu, cuda, ipu, xpu, mkldnn, opengl, opencl, ideep, hip, ve, fpga, maia, xla, lazy, vulkan, mps, meta, hpu, mtia, privateuseone device type at start of device string: CPU

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate)

for i in range(max_iters):
    if i%eval_iters == 0:
        print(f'step: {i} , loss: {losses}')
    xb, yb = get_batch('train')
    logits, loss  = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()
print(loss.item())

In [ ]:
context = torch.zeros(